In [36]:
from datasets import Dataset
import os
import requests
from google.cloud import speech
from tqdm import tqdm
from pydub import AudioSegment
from pathlib import Path
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\User\\Apviza\\Project 6\\propane-landing-397905-976c53b45983.json"
import tempfile
import shutil
from datasets import load_dataset
import Levenshtein
import pandas as pd
import string

In [37]:
# Define the root folder path
root_folder_path = "C:\\Users\\User\\Apviza\\Project 6\\Output"

# Initialize an empty list to store dataset dictionaries
dataset_dict_list = []

for root, _, _ in os.walk(root_folder_path):
    for subfolder in os.listdir(root):
        subfolder_path = os.path.join(root, subfolder)
        if os.path.isdir(subfolder_path):
            for sub_root, _, files in os.walk(subfolder_path):
                for file in files:
                    if file.endswith(".wav"):
                        audio_path = os.path.join(sub_root, file)
                        path_components = sub_root.split(os.path.sep)
                        dialect_region = path_components[-2]
                        speaker_id = path_components[-1]
                        dataset_dict = {
                            "audio": audio_path,
                            "dialect_region": dialect_region,
                            "speaker_id": speaker_id,
                            "audio_file": file
                        }
                        dataset_dict_list.append(dataset_dict)

data_dict = {
    "audio": [item["audio"] for item in dataset_dict_list],
    "dialect_region": [item["dialect_region"] for item in dataset_dict_list],
    "speaker_id": [item["speaker_id"] for item in dataset_dict_list],
    "audio_file": [item["audio_file"] for item in dataset_dict_list]
}

dataset = Dataset.from_dict(data_dict)

In [3]:
def convert_audio(audio_path, converted_path):
    audio = AudioSegment.from_wav(audio_path)
    # Convert to 16-bit PCM, 16 kHz sample rate
    audio = audio.set_frame_rate(16000).set_sample_width(2).set_channels(1)
    audio.export(converted_path, format="wav")


def transcribe_audio(converted_path):
    client = speech.SpeechClient()

    with open(converted_path, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="en-US",
    )

    response = client.recognize(config=config, audio=audio)

    transcription = ""
    for result in response.results:
        transcription += result.alternatives[0].transcript

    return transcription

In [ ]:
output_dir = "C:/Users/User/Apviza/Project 6/Output"
converted_audio_dir = "C:/Users/User/Apviza/Project 6/Converted_Audio"
examples = dataset

for example in tqdm(examples, desc="Transcribing audio", unit="audio"):
    audio_path = example["audio"]
    audio_file = os.path.basename(audio_path)

    # Extract the relevant information from the example dictionary
    dialect_region = example["dialect_region"]
    speaker_id = example["speaker_id"]

    # Create output directory structure
    output_path = Path(output_dir) / dialect_region / speaker_id
    output_path.mkdir(parents=True, exist_ok=True)

    converted_audio_path = Path(converted_audio_dir) / dialect_region / speaker_id
    converted_audio_path.mkdir(parents=True, exist_ok=True)

    converted_path = converted_audio_path / audio_file
    convert_audio(audio_path, converted_path)

    transcription = transcribe_audio(converted_path)

    output_file = output_path / f"Transcription_{audio_file}.txt"
    with open(output_file, "w") as f:
        f.write(transcription)

Transcribing audio:  50%|████▉     | 3114/6252 [56:47<57:00,  1.09s/audio]  

In [20]:
dataset = load_dataset("timit_asr",data_dir='C:\\Users\\User\\Apviza\\Project 6\\TIMIT')

In [23]:
output_dir = "C:/Users/User/Apviza/Project 6/Output"
examples = dataset["train"]
for example in tqdm(examples, desc="Copying Original Text", unit="txt"):
    text = example["text"]
    dialect_region = example["dialect_region"]
    speaker_id = example["speaker_id"]
    id_ = example["id"]
        
    # Create output directory structure
    output_path = Path(output_dir) / dialect_region / speaker_id
    
    # Check if the destination directory exists
    if output_path.exists() and output_path.is_dir():
        # Create the destination path for the new text file
        destination_text_path = output_path / f"{id_}.txt"
        
        # Write the content from the 'text' variable to the new text file
        with open(destination_text_path, "w") as f:
            f.write(text)

        


Copying Original Text: 100%|██████████| 4620/4620 [00:29<00:00, 159.29txt/s]


In [38]:
# Define the root folder path
root_folder_path = "C:\\Users\\User\\Apviza\\Project 6\\Output"

# Initialize a list to store dataset dictionaries
dataset_dict_list = []

for root, _, _ in os.walk(root_folder_path):
    for subfolder in os.listdir(root):
        subfolder_path = os.path.join(root, subfolder)
        if os.path.isdir(subfolder_path):
            for sub_root, _, files in os.walk(subfolder_path):
                audio_path = None
                synthtxt_path = None
                sourcetxt_path = None
                for file in files:
                    if file.endswith(".wav"):
                        audio_path = os.path.join(sub_root, file)
                        path_components = sub_root.split(os.path.sep)
                        dialect_region = path_components[-2]
                        speaker_id = path_components[-1]
                    elif file.endswith(".txt"):
                        txt_path = os.path.join(sub_root, file)
                        if file.startswith("Transcription_"):
                            synthtxt_path = txt_path
                        else:
                            sourcetxt_path = txt_path
                if audio_path is not None:
                    dataset_dict = {
                        "audio": audio_path,
                        "dialect_region": dialect_region,
                        "speaker_id": speaker_id,
                        "audio_file": os.path.basename(audio_path),
                        "synthtxt": synthtxt_path,
                        "synthtxt_file": os.path.basename(synthtxt_path) if synthtxt_path else None,
                        "sourcetxt": sourcetxt_path,
                        "sourcetxt_file": os.path.basename(sourcetxt_path) if sourcetxt_path else None
                    }
                    dataset_dict_list.append(dataset_dict)

data_dict = {
    "audio": [item["audio"] for item in dataset_dict_list],
    "dialect_region": [item["dialect_region"] for item in dataset_dict_list],
    "speaker_id": [item["speaker_id"] for item in dataset_dict_list],
    "audio_file": [item["audio_file"] for item in dataset_dict_list],
    "synthtxt": [item["synthtxt"] for item in dataset_dict_list],
    "synthtxt_file": [item["synthtxt_file"] for item in dataset_dict_list],
    "sourcetxt": [item["sourcetxt"] for item in dataset_dict_list],
    "sourcetxt_file": [item["sourcetxt_file"] for item in dataset_dict_list]
}

dataset = Dataset.from_dict(data_dict)

In [39]:
dataset[0]

{'audio': 'C:\\Users\\User\\Apviza\\Project 6\\Output\\DR1\\CJF0\\SX397.wav',
 'dialect_region': 'DR1',
 'speaker_id': 'CJF0',
 'audio_file': 'SX397.wav',
 'synthtxt': 'C:\\Users\\User\\Apviza\\Project 6\\Output\\DR1\\CJF0\\Transcription_SX397.wav.txt',
 'synthtxt_file': 'Transcription_SX397.wav.txt',
 'sourcetxt': 'C:\\Users\\User\\Apviza\\Project 6\\Output\\DR1\\CJF0\\SX397.txt',
 'sourcetxt_file': 'SX397.txt'}

In [40]:
def calculate_wer(reference_text, hypothesis_text):
    # Split the texts into words
    ref_words = reference_text.split()
    hyp_words = hypothesis_text.split()

    # Calculate the Levenshtein distance between the word sequences
    distance = Levenshtein.distance(ref_words, hyp_words)

    # Calculate the WER
    wer = distance / len(ref_words)

    return wer

def preprocess_text(text):
    # Remove punctuation and convert to lowercase
    translator = str.maketrans("", "", string.punctuation)
    cleaned_text = text.translate(translator).lower()
    return cleaned_text

In [41]:
examples = dataset

dialect_regions = []
speaker_ids = []
audio_files = []
wers = []

for example in tqdm(examples, desc="Word Error Rate", unit=" file"):
    synthtxt_path = example["synthtxt"]
    sourcetxt_path = example["sourcetxt"]

    # Load the content of the txt files
    with open(synthtxt_path, "r") as synthtxt_file:
        synthtxt = synthtxt_file.read().strip()
    
    with open(sourcetxt_path, "r") as sourcetxt_file:
        sourcetxt = sourcetxt_file.read().strip()

    # Preprocess the text
    synthtxt = preprocess_text(synthtxt)
    sourcetxt = preprocess_text(sourcetxt)

    # Calculate WER
    wer = calculate_wer(sourcetxt, synthtxt)
    
    # Append data to lists
    dialect_regions.append(example["dialect_region"])
    speaker_ids.append(example["speaker_id"])
    audio_files.append(example["audio_file"])
    wers.append(wer)
    
# Create a Pandas DataFrame
data = {
    "Dialect Region": dialect_regions,
    "Speaker ID": speaker_ids,
    "Audio File": audio_files,
    "WER": wers
}

df = pd.DataFrame(data)


Word Error Rate: 100%|██████████| 626/626 [00:00<00:00, 1089.71 file/s]


In [42]:
df = df.drop_duplicates(subset=["Dialect Region", "Speaker ID", "Audio File"])
df

,Dialect Region,Speaker ID,Audio File,WER
0,DR1,CJF0,SX397.wav,0.111111
1,DR1,CPM0,SX384.wav,0.000000
2,DR1,DAC0,SX91.wav,0.166667
3,DR1,DAW0,SX56.wav,0.000000
4,DR1,DML0,SX69.wav,0.000000
...,...,...,...,...
308,DR5,SKP0,SX378.wav,0.111111
309,DR5,SMM0,SX54.wav,0.000000
310,DR5,SMS1,SX64.wav,0.000000
311,DR5,TBW0,SX85.wav,0.166667


In [43]:
df_sorted = df.sort_values(by="WER", ascending=False)
df_sorted

,Dialect Region,Speaker ID,Audio File,WER
271,DR5,EGJ0,SX77.wav,0.666667
41,DR2,BJV0,SX77.wav,0.666667
9,DR1,GRL0,SX57.wav,0.625000
96,DR2,RJM0,SX58.wav,0.500000
157,DR3,LJD0,SX76.wav,0.500000
...,...,...,...,...
193,DR4,BAS0,SX397.wav,0.000000
84,DR2,MKF0,SX96.wav,0.000000
82,DR2,MGK0,SX62.wav,0.000000
81,DR2,MDS0,SX83.wav,0.000000


In [48]:
filtered_df = df[df["WER"] > 0]
filtered_df

,Dialect Region,Speaker ID,Audio File,WER
0,DR1,CJF0,SX397.wav,0.111111
2,DR1,DAC0,SX91.wav,0.166667
9,DR1,GRL0,SX57.wav,0.625000
10,DR1,JEB1,SX387.wav,0.375000
11,DR1,JSP0,SX84.wav,0.166667
...,...,...,...,...
302,DR5,RAV0,SX378.wav,0.111111
306,DR5,SDC0,SX52.wav,0.142857
307,DR5,SJG0,SX400.wav,0.142857
308,DR5,SKP0,SX378.wav,0.111111


In [47]:
target_dialect_region = "DR5"
target_speaker_id = "EGJ0"
target_audio_file = "SX77.wav"

for example in dataset:
    if (example["dialect_region"] == target_dialect_region and
        example["speaker_id"] == target_speaker_id and
        example["audio_file"] == target_audio_file):
        
        synthtxt_path = example["synthtxt"]
        sourcetxt_path = example["sourcetxt"]

        # Load and display the content of the synthtxt and sourcetxt files
        with open(synthtxt_path, "r") as synthtxt_file:
            synthtxt = synthtxt_file.read().strip()
            print("Synthesized Text:")
            print(synthtxt)
        
        with open(sourcetxt_path, "r") as sourcetxt_file:
            sourcetxt = sourcetxt_file.read().strip()
            print("Source Text:")
            print(sourcetxt)

Synthesized Text:
backups and bond with their musical instruments
Source Text:
Bagpipes and bongos are musical instruments.
Synthesized Text:
backups and bond with their musical instruments
Source Text:
Bagpipes and bongos are musical instruments.


In [49]:
print(df["WER"].mean())

0.10778060242916473


In [ ]:
use rnn, lstm for calssification model